In [1]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [2]:
# --------- User configurable settings ---------
DATA_TXT = "/kaggle/input/tinystories40k/tinystories_train40.txt"
TOKENIZER_FILE = "/kaggle/input/tinystories/tinystories_train.json"
MAX_CHARS = 40_000_000 # read at most 40 million characters
VOCAB_SIZE = 4096
BATCH_SIZE = 64
BLOCK_SIZE = 256
MAX_ITERS = 20000
EVAL_INTERVAL = 200
LEARNING_RATE = 3e-4
EVAL_ITERS = 50 # reduced for speed during eval
GRAD_CLIP = 1.0


# Model size reduced to better match small dataset
N_EMBD = 256
N_HEAD = 4
N_LAYER = 4
DROPOUT = 0.1

In [3]:
# Device (Kaggle friendly)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [4]:
TOKENIZER_FILE = "/kaggle/working/tokenizer.json"

# ---------------- Tokenizer (train once) ----------------
if not os.path.exists(TOKENIZER_FILE):
    print("Training tokenizer...")
    tokenizer = Tokenizer(BPE(unk_token='[UNK]'))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = BpeTrainer(
        vocab_size=VOCAB_SIZE,
        special_tokens=['[UNK]', '[CLS]', '[SEP]', '[PAD]', '[MASK]']
    )
    tokenizer.train([DATA_TXT], trainer)

    tokenizer.save(TOKENIZER_FILE)
    print("Tokenizer saved to", TOKENIZER_FILE)

else:
    print("Loading existing tokenizer...")
    tokenizer = Tokenizer.from_file(TOKENIZER_FILE)

vocab_size = tokenizer.get_vocab_size()
print("Vocab size:", vocab_size)

encode = lambda s: tokenizer.encode(s).ids
decode = lambda ids: tokenizer.decode(ids)


Training tokenizer...



Tokenizer saved to /kaggle/working/tokenizer.json
Vocab size: 4096


In [5]:
# ------------ Load data ------------
with open(DATA_TXT, 'r', encoding='utf-8') as f:
    text = f.read(MAX_CHARS)


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
# --------------- helpers ---------------
def get_batch(split):
    data_src = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_src) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data_src[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data_src[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x.to(device), y.to(device)

In [7]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

In [8]:
# --------------- Rotary embeddings ---------------
class RotaryPositionalEmbeddings(nn.Module):
    def __init__(self, head_dim, max_seq_len=BLOCK_SIZE):
        super().__init__()
        # standard RoPE inverse frequency
        inv_freq = 10000 ** (-torch.arange(0, head_dim, 2).float() / head_dim)
        positions = torch.arange(max_seq_len).float().unsqueeze(1)
        angles = positions * inv_freq.unsqueeze(0) # (max_seq_len, head_dim/2)
        # store cos and sin as buffers so they move with module.to(device)
        self.register_buffer('cos_cached', angles.cos())
        self.register_buffer('sin_cached', angles.sin())
    
    
    def forward(self, seq_len):
        # return slices shaped (seq_len, head_dim//2)
        return self.cos_cached[:seq_len, :], self.sin_cached[:seq_len, :]

In [9]:
def apply_rotary_emb(x, cos, sin):
    # x: (B, T, head_dim)
    d = x.shape[-1] // 2
    x1 = x[..., :d]
    x2 = x[..., d:]
    # cos and sin are (T, d) need to broadcast to (B, T, d)
    # make shapes explicit for readability
    cos = cos.unsqueeze(0)
    sin = sin.unsqueeze(0)
    out1 = x1 * cos - x2 * sin
    out2 = x1 * sin + x2 * cos
    return torch.cat([out1, out2], dim=-1)

In [10]:
# --------------- Attention and Transformer blocks ---------------
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.key = nn.Linear(N_EMBD, head_size, bias=False)
        self.query = nn.Linear(N_EMBD, head_size, bias=False)
        self.value = nn.Linear(N_EMBD, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))
        self.dropout = nn.Dropout(DROPOUT)
        self.rotary = RotaryPositionalEmbeddings(head_size, max_seq_len=BLOCK_SIZE)
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # get cos and sin for sequence length T
        cos, sin = self.rotary(T)
        q = apply_rotary_emb(q, cos, sin)
        k = apply_rotary_emb(k, cos, sin)
        # scaled dot product
        wei = q @ k.transpose(-2, -1) * (self.head_size ** -0.5)
        mask = self.tril[:T, :T].to(wei.device)
        wei = wei.masked_fill(mask == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, N_EMBD)
        self.dropout = nn.Dropout(DROPOUT)
    
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.GELU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(DROPOUT),
        )
    
    
    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [11]:
# --------------- GPT Model ---------------
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, N_EMBD)
        self.blocks = nn.Sequential(*[Block(N_EMBD, n_head=N_HEAD) for _ in range(N_LAYER)])
        self.ln_f = nn.LayerNorm(N_EMBD)
        self.lm_head = nn.Linear(N_EMBD, vocab_size)
        self.apply(self._init_weights)
    
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        x = tok_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            return logits, None
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [12]:
# --------------- instantiate model, optimizer, scaler ---------------
model = GPTLanguageModel()
model = model.to(device)
print("Model parameters: ", sum(p.numel() for p in model.parameters()) / 1e6, "M")


optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
import torch.amp

scaler = torch.amp.GradScaler(enabled=(device == 'cuda'))


Model parameters:  5.257728 M


In [13]:
for it in range(MAX_ITERS):
    if it % EVAL_INTERVAL == 0 or it == MAX_ITERS - 1:
        losses = estimate_loss(model)
        print(f"step {it}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch('train')
    
    optimizer.zero_grad(set_to_none=True)
    
    # mixed precision when on CUDA
    with torch.amp.autocast(device_type='cuda', enabled=(device == 'cuda')):
        _, loss = model(xb, yb)
    
    if device == 'cuda':
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        scaler.step(optimizer)
        scaler.update()
    else:
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        optimizer.step()


step 0: train loss 8.3753, val loss 8.3735
step 200: train loss 3.6616, val loss 3.6209
step 400: train loss 3.1982, val loss 3.1847
step 600: train loss 2.9541, val loss 2.9335
step 800: train loss 2.7873, val loss 2.7636
step 1000: train loss 2.6557, val loss 2.6553
step 1200: train loss 2.5677, val loss 2.5682
step 1400: train loss 2.4832, val loss 2.4868
step 1600: train loss 2.4159, val loss 2.4289
step 1800: train loss 2.3550, val loss 2.3678
step 2000: train loss 2.3139, val loss 2.3503
step 2200: train loss 2.2620, val loss 2.3031
step 2400: train loss 2.2271, val loss 2.2655
step 2600: train loss 2.1903, val loss 2.2418
step 2800: train loss 2.1755, val loss 2.2241
step 3000: train loss 2.1407, val loss 2.1872
step 3200: train loss 2.1085, val loss 2.1765
step 3400: train loss 2.0882, val loss 2.1736
step 3600: train loss 2.0775, val loss 2.1477
step 3800: train loss 2.0524, val loss 2.1327
step 4000: train loss 2.0283, val loss 2.1067
step 4200: train loss 2.0073, val loss 2.

In [14]:
# --------------- generation example ---------------
print("Generating story...")
start_prompt = "Once upon a time"
context = torch.tensor([encode(start_prompt)], dtype=torch.long, device=device)
with torch.no_grad():
    generated = model.generate(context, max_new_tokens=300, temperature=1.0, top_k=50)[0].tolist()
print(decode(generated))

Generating story...
Once upon a time there was a little girl called Sarah . She was very obedient and always listened to her parents . One day , her parents told her that she would always listen to her . Sarah had to do all the cho o they could do . On this day , she couldn ' t believe it ! At first , it was so strange ly dirty . Sarah had to be careful , so she picked up a pink flower and gently picked it carefully . The flower was so special . Sarah stayed put , but it remain ed the best time - until she got tired from going to a nap . Soon the sun came in the sky , and Sarah was getting better . She yaw ned in the air , and then finally closed her eyes . But Sarah was still very excited . She realized she could play so own , she just had to listen to her mom . â € œ Mommy , â € Sarah said sadly . â € œ Little W anda , that sounds like she ' s boun cy castle . â € Her mom smiled too . " Yes , it was . Letâ €™ s do !" Sarah and She grabbed her ball and threw it to the dog . The dog , 

In [15]:
# --------------- optional: save model checkpoint ---------------
CKPT_PATH = "tinystories_gpt.pt"
try:
    torch.save({'model_state_dict': model.state_dict(), 'tokenizer': TOKENIZER_FILE}, CKPT_PATH)
    print("Saved checkpoint to", CKPT_PATH)
except Exception as e:
    print("Could not save checkpoint:", e)

Saved checkpoint to tinystories_gpt.pt
